In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook
import json
import os
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt

utils = Utils()
import time

2023-12-04 18:18:01.355861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 18:18:01.397262: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
def create_dataframe(experiment):
    datagenerator = DataGenerator()

    noise_type = (
        None if experiment["noise_type"] == "normal" else experiment["noise_type"]
    )  # irrelevant,duplicated
    anomaly_type = (
        None if experiment["anomaly_type"] == "normal" else experiment["anomaly_type"]
    )  # cluster,global,local,dependency

    datagenerator.dataset = experiment["dataset"]
    data = datagenerator.generator(
        la=1.00,
        realistic_synthetic_mode=anomaly_type,
        noise_type=noise_type,
        noise_ratio=float(experiment["noise_ratio"]),
        stdscale=True,
        minmax=False,
    )

    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)

    return Y_train, Y_test, lb_train, lb_test

In [40]:
EXPERIMENTS_FILE = "experiments/complete/000_datasets_01_47_normal_00_exps.json"
with open(EXPERIMENTS_FILE) as file:
    experiments = json.load(file)
df = pd.DataFrame(experiments)
df = df[df.dataset == "34_smtp"]
experiments = df.to_dict(orient="records")

In [41]:
Y_train, Y_test, lb_train, lb_test = create_dataframe(experiments[0])
lb_train.sum()/len(lb_train), lb_test.sum()/len(lb_test)

subsampling for dataset 34_smtp...


(tensor(0.0001), tensor(0.0003))

In [12]:
success_experiments = []
failed_experiments = []
for experiment in tqdm_notebook(experiments):
    Y_train, Y_test, lb_train, lb_test = create_dataframe(experiment)
    Y_val, Y_test, lb_val, lb_test = train_test_split(
        Y_test, lb_test, test_size=0.50, random_state=42
    )

    if experiment["loss"] == "normal":
        idx_n = np.where(lb_train == 0)[0]
        Y_train_normal = Y_train[idx_n]
        lb_train_normal = lb_train[idx_n]

    n_train = len(Y_train_normal)
    data_dim = Y_train_normal.shape[1]
    kernel = experiment["kernel"]
    latent_dim = int(experiment["latent_dim"])
    nn_layers = tuple(map(int, experiment["layers"].split(",")))
    n_inducing = int(experiment["n_inducing"])
    n_epochs = int(experiment["n_epochs"])
    lr = float(experiment["learning_rate"])
    batch_size = int(experiment["batch_size"])

    # print(experiment)

    try:
        gplvm = AD_GPLVM(
            latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size, kernel
        )

        # Fitting the Model
        train_start_time = time.time()
        gplvm.fit(Y_train_normal)
        train_end_time = time.time()

        # Validating the Model
        val = []
        for i in range(100):
            score = gplvm.predict_score(Y_val)
            val.append(score)
        validation_score = np.mean(val, axis=0)
        # validation_score = gplvm.calculate_train_elbo(Y_train_normal)

        # Results
        pred_start_time = time.time()
        test_score = gplvm.predict_score(Y_test)
        pred_end_time = time.time()

        # Save Metrics
        metrics = utils.metric(y_true=lb_val, y_score=validation_score)
        validation_metrics = utils.metric(y_true=lb_test, y_score=test_score)
        experiment["negative_elbo"] = validation_score.sum()  # validation_score.sum()
        experiment["train_loss_curve"] = gplvm.loss_list
        experiment["val_auc_roc"] = validation_metrics["aucroc"]
        experiment["val_auc_pr"] = validation_metrics["aucpr"]
        experiment["test_auc_roc"] = metrics["aucroc"]
        experiment["test_auc_pr"] = metrics["aucpr"]
        experiment["training_time"] = train_end_time - train_start_time
        experiment["inference_time"] = pred_end_time - pred_start_time

        # Reconstrucao
        Y_val_recon, Y_val_recon_covar = gplvm.model.reconstruct_y(Y_val)
        experiment["val_reconstruct_error"] = float(utils.rmse(Y_val, Y_val_recon.T))

        utils.save_experiment(experiment)

    except Exception as error:
        print("An exception occurred:", error)
        experiment["test_auc_roc"] = 0.0
        experiment["test_auc_pr"] = 0.0
        utils.save_experiment(experiment)

/tmp/ipykernel_16938/1405413897.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for experiment in tqdm_notebook(experiments):


  0%|          | 0/72 [00:00<?, ?it/s]

subsampling for dataset 34_smtp...


/home/jovyan/work/new_aeb_gplvm.py:319: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  klu_expanded = ll_shape.T.add_(klu).sum(-1).T.div((self.n_train))


An exception occurred: Only one class present in y_true. ROC AUC score is not defined in that case.
subsampling for dataset 34_smtp...


KeyboardInterrupt: 